In [1]:
import numpy as np
import pandas as pd


# Primers of advance pandas usage

### SQL - like things
* [JOIN](#join)  / [merge](#merge)
* [COUNT DISTINCT / nunique](#nunique)
* [GROUP BY / aggregate](#Group by and aggregate)

### Machine Learning
* [One hot encoding](#get_dummies)

### Big data
* [chunk](#chunk)

### Excel-like things
* [Operation between rows](#row-operation)

# SQL - like things

## join

In [3]:
caller = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3', 'K4', 'K5'],
                          'A': ['A0', 'A1', 'A2', 'A3', 'A4', 'A5']})
other = pd.DataFrame({'key_A': ['K0', 'K1', 'K2'],
                       'B': ['B0', 'B1', 'B2']})

In [4]:
caller

,A,key
0,A0,K0
1,A1,K1
2,A2,K2
3,A3,K3
4,A4,K4
5,A5,K5


In [5]:
other

,B,key_A
0,B0,K0
1,B1,K1
2,B2,K2


### to join `ON A.key = B.key_A `  :

In [6]:
caller.join(other.set_index('key_A'), on='key', lsuffix='_caller', rsuffix='_other')
# suffix only added when there are columns with same name

,A,key,B
0,A0,K0,B0
1,A1,K1,B1
2,A2,K2,B2
3,A3,K3,NaN
4,A4,K4,NaN
5,A5,K5,NaN


## merge

(merge is the more flexible form of join)

In [ ]:
caller.merge(other, left_on= 'key', right_on='key_A', how='left')


## nunique
[see here](https://stackoverflow.com/questions/15411158/pandas-countdistinct-equivalent)

`.nunique()`

## Group by and aggregate

In [15]:
df = pd.DataFrame({'A': ['a', 'b', 'a', 'a', 'b', 'a'], 'B': ['b', 'a', 'c', 'a', 'a', 'c'], 'C': [1, 2, 3, 4, 4, 3]})
df

,A,B,C
0,a,b,1
1,b,a,2
2,a,c,3
3,a,a,4
4,b,a,4
5,a,c,3


In [29]:
f = {'B': ['nunique', 'count'], 'C': ['sum']}
df_agg = df.groupby('A').agg(f)
df_agg

C       B      
  sum nunique count
A                  
a  11       3     4
b   6       1     2

#### flatten the resulting multiindex columns

[ref.](https://stackoverflow.com/questions/14507794/python-pandas-how-to-flatten-a-hierarchical-index-in-columns)

In [26]:
def flat_column(df):
    new = []
    for cc in df_agg.columns.values:
        new.append('_'.join(cc))
    return new

In [30]:
df_agg.columns = flat_column(df_agg)
df_agg

,C_sum,B_nunique,B_count
A,,,
a,11,3,4
b,6,1,2


# Machine Learning 

## get_dummies
## One hot encoding

In [3]:
df = pd.DataFrame({'A': ['a', 'b', 'a'], 'B': ['b', 'a', 'c'], 'C': [1, 2, 3]})
df

,A,B,C
0,a,b,1
1,b,a,2
2,a,c,3


In [4]:
pd.get_dummies(df, columns =['A','C'], prefix=['A', 'C'])

,B,A_a,A_b,C_1,C_2,C_3
0,b,1,0,1,0,0
1,a,0,1,0,1,0
2,c,1,0,0,0,1


# Big data

## Chunk

In [22]:
df = pd.DataFrame(np.arange(1000), columns=['id'])
df.to_csv('data/mock.csv')
del df

In [20]:
# read in only the selected vehicle data
tp = pd.read_csv('data/mock.csv', index_col=[0], iterator=True, chunksize=100)
sub_df = pd.DataFrame()
sub_df = pd.concat([chunk[chunk['id'] % 3 == 0] for chunk in tp])

## Excel-like things

<a id='row-operation'></a>

### Operation between rows

In [40]:
df = pd.DataFrame({'A': ['a', 'b', 'a'], 'start': [0,1, 3], 'end': [1, 2, 3]})
df.index = [1,5,8]
df

,A,end,start
1,a,1,0
5,b,2,1
8,a,3,3


In [41]:
df.reset_index(inplace=True)
df['C'] = np.nan # initialize
for i in range(1, len(df)):
    df.loc[i, 'C'] = df.loc[i, 'start'] - df.loc[i-1, 'end']
    

In [42]:
df

,index,A,end,start,C
0,1,a,1,0,NaN
1,5,b,2,1,0.0
2,8,a,3,3,1.0
